In [8]:
from pathlib import Path
import wandb
import torch

model_id = "model-1tgv4b7q:v0"
model_path = Path("artifacts") / model_id

api = wandb.Api()
artifact = api.artifact("ain-space/gaia/" + model_id, type="model")
if model_path.exists():
    print("Model already downloaded")
else:
    artifact_dir = artifact.download(model_path)

ckpt = model_path / "model.ckpt"
print("Loading checkpoint from", ckpt)
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")

wandb:   1 of 1 files downloaded.  


Loading checkpoint from artifacts/model-1tgv4b7q:v0/model.ckpt


In [10]:
from spherinator.models import yaml2model

model = yaml2model("../experiments/gaia_vae_8.yaml")
output = model(torch.randn(1, 1, 343))

/home/doserbd/git/Spherinator/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
/home/doserbd/git/Spherinator/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.


In [11]:
import torch

ckpt = artifact_dir + "/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
checkpoint

{'epoch': 920,
 'global_step': 218277,
 'pytorch-lightning_version': '2.5.0.post0',
 'state_dict': OrderedDict([('encoder.cnn.0.0.0.weight',
               tensor([[[-1.5134e+00,  1.4746e-02,  4.8442e-02, -2.5839e-01, -5.6778e-02,
                          1.1472e-01,  2.5260e-01]],
               
                       [[ 1.0116e-01,  2.1369e-01, -3.4065e-01, -4.3153e-01,  4.4639e-01,
                          6.3945e-01, -7.0910e-01]],
               
                       [[-9.2153e-01, -9.0669e-02,  5.5337e-01, -1.4462e-03,  1.2412e-01,
                          1.2252e-01, -6.7894e-02]],
               
                       [[ 6.8837e-01,  4.6988e-01, -1.1482e-01, -9.6383e-02, -1.1337e-01,
                         -4.8419e-01, -5.7664e-01]],
               
                       [[ 6.3551e-01, -1.8928e-01, -3.5999e-01, -1.5950e-01,  1.8171e-01,
                          8.1863e-02, -8.4338e-01]],
               
                       [[ 6.3423e-01, -3.2672e-01, -5.0468e-01, 

In [12]:
model.load_state_dict(checkpoint["state_dict"])
model.eval()

input = torch.randn(10, 1, 343)
model(input)

((tensor([[ 0.2525, -0.1626,  0.0576,  0.1054, -0.0112, -0.1675,  0.3468,  0.1140,
            0.1191, -0.2487,  0.2584, -0.1263, -0.0710,  0.0447, -0.2074,  0.4610,
            0.2571, -0.0873,  0.3680,  0.3217],
          [ 0.2672, -0.1781,  0.0675,  0.2064, -0.0425, -0.1651,  0.3132,  0.1192,
            0.1095, -0.1476,  0.2248, -0.0732,  0.0585, -0.0147, -0.1401,  0.5755,
            0.2467, -0.0635,  0.2950,  0.3351],
          [ 0.2079, -0.1796,  0.0088,  0.2426, -0.0478, -0.1418,  0.2811,  0.1147,
            0.1413, -0.1707,  0.2347, -0.0890,  0.0207,  0.0939, -0.2590,  0.5609,
            0.1819,  0.0732,  0.2001,  0.4117],
          [ 0.2756, -0.0936,  0.0463,  0.1524,  0.0300, -0.0805,  0.3870,  0.1451,
            0.0431, -0.1646,  0.2639, -0.0926, -0.0774,  0.0253, -0.1560,  0.5149,
            0.3547,  0.0399,  0.3008,  0.3010],
          [ 0.2232, -0.1835,  0.1419,  0.1877, -0.0185, -0.1545,  0.3389,  0.0935,
            0.0597, -0.1850,  0.1625, -0.0905, -0.0297,  0.05

In [15]:
import os

export_path = "gaia-calibrated-v1"
os.makedirs(export_path, exist_ok=True)

export_options = torch.onnx.ExportOptions(dynamic_shapes=False)
onnx = torch.onnx.dynamo_export(
    model.variational_encoder,
    torch.randn(256, 1, 343, device="cpu"),
    export_options=export_options,
)
onnx.save(os.path.join(export_path, "encoder.onnx"))

onnx = torch.onnx.dynamo_export(
    model.decoder, torch.randn(256, 20, device="cpu"), export_options=export_options
)
onnx.save(os.path.join(export_path, "decoder.onnx"))

/home/doserbd/git/Spherinator/.venv/lib/python3.12/site-packages/torch/onnx/_internal/_exporter_legacy.py:101: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/doserbd/git/Spherinator/.venv/lib/python3.12/site-packages/torch/onnx/_internal/fx/passes/readability.py:52: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  new_node = self.module.graph.get_attr(normalized_name)
/home/doserbd/git/Spherinator/.venv/lib/python3.12/site-packages/torch/fx/graph.py:1801: UserWarning: Node encoder_cnn_0_0_1_running_mean target encoder/cnn/0/0/1/running_mean encoder/cnn/0/0/1/running_mean of  does not reference an nn.Module, n

Applied 2 of general pattern rewrite rules.


/home/doserbd/git/Spherinator/.venv/lib/python3.12/site-packages/torch/onnx/_internal/fx/passes/readability.py:52: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  new_node = self.module.graph.get_attr(normalized_name)
/home/doserbd/git/Spherinator/.venv/lib/python3.12/site-packages/torch/fx/graph.py:1801: UserWarning: Node fc_2_running_mean target fc/2/running_mean fc/2/running_mean of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/home/doserbd/git/Spherinator/.venv/lib/python3.12/site-packages/torch/fx/graph.py:1801: UserWarning: Node fc_2_running_var target fc/2/running_var fc/2/running_var of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr

In [17]:
import onnxruntime as ort

ort_session = ort.InferenceSession(os.path.join(export_path, "encoder.onnx"))

2025-03-05 20:15:36.238697397 [W:onnxruntime:, graph.cc:4285 CleanUnusedInitializersAndNodeArgs] Removing initializer 'ord_5'. It is not used by any node and should be removed from the model.


In [ ]:
#!rsync gaia-calibrated-v0 space:/var/www/html/space/models/